In [ ]:
!pip install transformers torch datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset


dataset = load_dataset('csv', delimiter='\t', data_files={'train': '/content/drive/MyDrive/Colab Notebooks/Fake News Generation/NewsCategoryDataset.csv'})


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset['train'][1]

{'Unnamed: 0': 1,
 'headline': 'american airlines flyer charged, banned for life after punching flight attendant on video',
 'category': 'U.S. NEWS'}

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    text = [f"{category}: {headline}" for category, headline in zip(examples['category'], examples['headline'])]
    return tokenizer(text, truncation=True, padding='max_length', max_length=50)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets['train'].set_format("pt", ['input_ids','attention_mask'])
# tokenized_datasets['train'].remove_columns(["Unnamed: 0", "headline", "category"])
# tokenized_datasets['train'] = tokenized_datasets['train'].add_column("labels", tokenized_datasets['train']['input_ids'])

model = GPT2LMHeadModel.from_pretrained(model_name).to("cuda")

for param in model.parameters():
    param.requires_grad = False

# Assuming GPT-2 model
for i in [-1, -2]:  # Last 2 blocks
    for param in model.transformer.h[i].parameters():
        param.requires_grad = True

# Enable training for the LM head
for param in model.lm_head.parameters():
    param.requires_grad = True

In [ ]:
tokenized_datasets['train'][0]

{'input_ids': tensor([   52,    13,    50,    13, 28840,    25,   625,   604,  1510, 45630,
           504,  4836,   510, 27409,   329,   267,  9383,  1313,    12, 16793,
           276, 39849,   312, 47949, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0])}

In [ ]:
from torch.utils.data import DataLoader

# Example: Create DataLoader (you may need to adjust parameters)
train_dataloader = DataLoader(tokenized_datasets['train'].remove_columns(["Unnamed: 0", "headline", "category"]), shuffle=True, batch_size=4)


In [ ]:
import numpy as np
cnt = 0
for x in train_dataloader:
  if (cnt > 5):
    break
  cnt += 1
  for key, value in x.items():

    print(key, value)
  # print(x.items())

input_ids [tensor([ 8845,  3525, 45346,    51]), tensor([46833,  1137, 44180,  3861]), tensor([28840, 30339,    25, 18697]), tensor([   25, 10979,   257,    25]), tensor([  661,    25, 42055, 28368]), tensor([ 389, 3879,  286,  286]), tensor([16318,  1443,   640,   616]), tensor([ 1497,   319, 50256,  1946]), tensor([  416,   916, 50256, 17781]), tensor([  428,  6713, 50256,    25]), tensor([ 5003,   423, 50256,  3326]), tensor([  959,   257, 50256,  4450]), tensor([  338,  8780, 50256,    11]), tensor([18304,  9912, 50256, 14841]), tensor([  291,  3275, 50256,  6413]), tensor([13777,   329, 50256,    11]), tensor([ 3809, 21628, 50256,   290]), tensor([50256, 15927, 50256, 37735]), tensor([50256, 19997, 50256,  2616]), tensor([50256, 50256, 50256, 50256]), tensor([50256, 50256, 50256, 50256]), tensor([50256, 50256, 50256, 50256]), tensor([50256, 50256, 50256, 50256]), tensor([50256, 50256, 50256, 50256]), tensor([50256, 50256, 50256, 50256]), tensor([50256, 50256, 50256, 50256]), tenso

In [ ]:
from transformers import GPT2LMHeadModel, AdamW


optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
tokenized_datasets['train'][1]

{'input_ids': tensor([   52,    13,    50,    13, 28840,    25, 45630,   272, 26225, 42464,
          5047,    11,  9301,   329,  1204,   706, 29044,  5474, 31277,   319,
          2008, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0])}

In [ ]:
import torch
from tqdm import tqdm
batch_size = 4

# Set the model to training mode
model.train()

# Fine-tuning loop
for epoch in range(1):
    cnt = 0
    for batch in tqdm(train_dataloader):


        inputs = {key: torch.Tensor(value).to("cuda") for key, value in batch.items()}
        inputs['labels'] = inputs['input_ids']

        # batch = {
        #     'input_ids': torch.stack(input_ids).to("cuda"),
        #     'attention_mask': torch.stack(attention_mask).to("cuda"),
        #     'labels': torch.stack(input_ids).to("cuda"),
        # }
        # print(batch)
        # Forward pass
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()



        if (cnt % 100 == 0):
            print(loss)

        cnt += 1

        optimizer.step()
        optimizer.zero_grad()

  0%|          | 3/52382 [00:00<1:18:52, 11.07it/s]

tensor(1.6005, device='cuda:0', grad_fn=<NllLossBackward0>)


  0%|          | 103/52382 [00:09<1:00:15, 14.46it/s]

tensor(1.5162, device='cuda:0', grad_fn=<NllLossBackward0>)


  0%|          | 203/52382 [00:15<56:22, 15.43it/s]

...

100%|█████████▉| 52202/52382 [1:01:42<00:12, 13.86it/s]

tensor(0.9324, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|█████████▉| 52304/52382 [1:01:49<00:05, 14.10it/s]

tensor(1.2242, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 52382/52382 [1:01:55<00:00, 14.10it/s]


In [ ]:
def generate_headline(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    output = model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, do_sample=True)
    headline = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return headline

prompt = "WORLD NEWS:"
for _ in range(5):
  print(generate_headline(prompt))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 saudi arabia’s prime minister has a message for iraqi president


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 trump’s visit to the u.s. is 'a disaster'


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 u.s. accuses russia of supporting isis in syria


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 saudi arabia says it’s ready to take action against trump
 the world’s most dangerous countries


In [ ]:
model

In [ ]:
torch.save(model.state_dict(), "gpt2_start_weights")

In [ ]:
model1 = GPT2LMHeadModel.from_pretrained(model_name)
model1.load_state_dict(torch.load("gpt2_start_weights"))


<All keys matched successfully>

In [ ]:
def generate_headline_1(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    output = model1.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2, early_stopping=True, do_sample=True)
    headline = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return headline

prompt = "WORLD NEWS:"
for _ in range(5):
  print(generate_headline(prompt))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 britain says it's ready to leave the u.s.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 north korea’s kim jong un'ye says he'll resign if u.s. sanctions don't work


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 north korea's kim jong un says it's time to give up its nuclear weapons


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 saudi arabia's president resigns over trump comments
 u.s. troops in afghanistan launch surprise attack on ukraine
